In [2]:
from world import CarlaWorld
from ego_vehicle import EgoVehicle
import yaml
import cv2
import numpy as np
import glob
import os
import sys
import matplotlib.pyplot as plt
import open3d as o3d
import time
import math

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass
import carla

try:
    import pygame
    from pygame.locals import K_ESCAPE
    from pygame.locals import K_q
except ImportError:
    raise RuntimeError('cannot import pygame, make sure pygame package is installed')

from utils import process_rgb_image, retrive_data, process_depth_image, process_sem_seg_image, process_point_cloud, add_open3d_axis, is_empty, capture_data
from pygame_display import DisplayManager
from bounding_box import ClientSideBoundingBoxes
lines = [[0, 1], [1, 2], [2, 3], [3, 0],
         [4, 5], [5, 6], [6, 7], [7, 4],
         [0, 4], [1, 5], [2, 6], [3, 7]]
with open('cfg\\vehicle_cfg.yaml', 'r') as f:
    vehicle_cfg = yaml.safe_load(f) 
    
with open('cfg\\config.yaml', 'r') as f:
    cfg = yaml.safe_load(f) 
    
carla_world = CarlaWorld(cfg)

bp_lib = carla_world.world.get_blueprint_library()
ego_vehicle = EgoVehicle(bp_lib, vehicle_cfg)
ego_vehicle.spwan_ego_vehicle(carla_world.world)
display_man = None
if cfg['sensor_preview']:
    grid_size = [math.ceil(ego_vehicle.num_cameras/3), 3]
    display_man = DisplayManager(grid_size, window_size=[1280, 720])

ego_vehicle.sensor_setup(carla_world.world, display_man, enable_lidar_vis=True)
static_bboxes = carla_world.world.get_level_bbs(carla.CityObjectLabel.Car)
carla_world.spawn_actors()
ego_vehicle.ego_vehicle.set_autopilot(True)

carla_world.set_synchronous()

out_dir = cfg['out_dir']
capture_frequency = cfg['capture_frequency']
simulation_frequency = cfg['fps']
delta_tick = int(simulation_frequency/capture_frequency)
assert delta_tick > 0, "please reduce capture_frequency"
frame = 0
call_exit = False
time.sleep(10.0)
## Main loop
while True:
    data = {}
    frame_id = carla_world.tick()    

    vehicles = carla_world.world.get_actors().filter('vehicle.*')
    
    # bounding_boxes = ClientSideBoundingBoxes.get_bounding_boxes(ego_vehicle.ego_vehicle, vehicles, ego_vehicle.sensors['lidar1'], additional_bb=static_bboxes)
    # for box in bounding_boxes:
    #     box[0,:] *= -1
    if display_man:
        for sensor in ego_vehicle.sensors:
            if sensor.sensor_type == 'RGBCamera':
                rgb, depth, sem_seg = sensor.retrive_data(frame_id, 2.0)
                
            if sensor.sensor_type == 'LiDAR':
                point_cloud = sensor.retrive_data(frame_id, 2.0)
                
    pygame.display.flip()

    frame += 1
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            call_exit = True
        elif event.type == pygame.KEYDOWN:
            if event.key == K_ESCAPE or event.key == K_q:
                call_exit = True
                break

    if call_exit:
        pygame.display.quit()
        for sensor in ego_vehicle.sensors:
            if sensor.sensor_type == 'LiDAR' and sensor.vis:
                sensor.vis.destroy_window()
                
        break
    
    # if frame == 100:
    #     break

carla_world.restore()
carla_world.destroy_actors()


rgb time 37.235755544155836
rgb time 37.235755544155836
rgb time 37.235755544155836
rgb time 37.235755544155836
rgb time 37.235755544155836
rgb time 37.235755544155836
rgb time 37.33575554564595
rgb time 37.33575554564595
rgb time 37.33575554564595
rgb time 37.33575554564595
rgb time 37.33575554564595
rgb time 37.33575554564595
rgb time 37.43575554713607
rgb time 37.43575554713607
rgb time 37.43575554713607
rgb time 37.43575554713607
rgb time 37.43575554713607
rgb time 37.43575554713607
rgb time 37.535755548626184
rgb time 37.535755548626184
rgb time 37.535755548626184
rgb time 37.535755548626184
rgb time 37.535755548626184
rgb time 37.535755548626184
rgb time 37.6357555501163
rgb time 37.6357555501163
rgb time 37.6357555501163
rgb time 37.6357555501163
rgb time 37.6357555501163
rgb time 37.6357555501163
rgb time 37.73575555160642
rgb time 37.73575555160642
rgb time 37.73575555160642
rgb time 37.73575555160642
rgb time 37.73575555160642
rgb time 37.73575555160642
rgb time 37.8357555530

In [ ]:
carla_world.restore()
carla_world.destroy_actors()


In [ ]:
num_cameras = 6
import math
grid_size = [math.ceil(num_cameras/3), 3]
grid_size

In [ ]:
carla_world.world.get_actor(carla.CityObjectLabel.Car)

In [ ]:
import cv2
import matplotlib.pyplot as plt
from pygame_display import DisplayManager
import pygame
from pygame.locals import K_ESCAPE
from pygame.locals import K_q

img = cv2.imread(r"D:\rbe\lara\renders\untitled1.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
display = DisplayManager([1,3], [1280, 720])
call_exit = False
while True:
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            call_exit = True
        elif event.type == pygame.KEYDOWN:
            if event.key == K_ESCAPE or event.key == K_q:
                call_exit = True
                break
    if call_exit:
        break
    
    resize = display.get_display_size()
    img = cv2.resize(img, resize)
    surface = pygame.surfarray.make_surface(img)
    offset = display.get_display_offset([0,3])
    display.display.blit(surface, offset)

plt.imshow(img)

In [ ]:
import numpy as np
r = 1
degs = [0, 60, 120, 180, 240, 300]

for deg in degs:
    x = np.cos(np.deg2rad(deg))
    y = np.sin(np.deg2rad(deg))
    print(x,y)